# Kapitel 7: Byg chatapplikationer
## Github Models API Kom godt i gang

Denne notesbog er tilpasset fra [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst), som indeholder notesbøger, der giver adgang til [Azure OpenAI](notebook-azure-openai.ipynb) tjenester.


# Oversigt  
"Store sprogmodeller er funktioner, der omsætter tekst til tekst. Givet en inputstreng af tekst forsøger en stor sprogmodel at forudsige, hvilken tekst der kommer bagefter"(1). Denne "quickstart"-notebook vil introducere brugere til overordnede LLM-begreber, centrale pakkekrav for at komme i gang med AML, en blid introduktion til promptdesign og flere korte eksempler på forskellige anvendelsestilfælde.


## Indholdsfortegnelse  

[Oversigt](../../../../07-building-chat-applications/python)  
[Sådan bruger du OpenAI Service](../../../../07-building-chat-applications/python)  
[1. Opret din OpenAI Service](../../../../07-building-chat-applications/python)  
[2. Installation](../../../../07-building-chat-applications/python)    
[3. Legitimationer](../../../../07-building-chat-applications/python)  

[Anvendelsesmuligheder](../../../../07-building-chat-applications/python)    
[1. Opsummér tekst](../../../../07-building-chat-applications/python)  
[2. Klassificér tekst](../../../../07-building-chat-applications/python)  
[3. Generér nye produktnavne](../../../../07-building-chat-applications/python)  
[4. Finjustér en klassifikator](../../../../07-building-chat-applications/python)  

[Referencer](../../../../07-building-chat-applications/python)


### Byg din første prompt  
Denne korte øvelse giver en grundlæggende introduktion til at indsende prompts til en model i Github Models for en simpel opgave: "opsummering".


**Trin**:  
1. Installer biblioteket `azure-ai-inference` i dit python-miljø, hvis du ikke allerede har gjort det.  
2. Indlæs standard hjælpebiblioteker og opsæt Github Models-legitimationsoplysninger.  
3. Vælg en model til din opgave  
4. Opret en simpel prompt til modellen  
5. Send din forespørgsel til model-API'en!


### 1. Installer `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

### 2. Importer hjælperbiblioteker og opret legitimationsoplysninger


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Find den rette model  
GPT-3.5-turbo eller GPT-4 modellerne kan forstå og generere naturligt sprog.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Promptdesign  

"Magien ved store sprogmodeller er, at ved at blive trænet til at minimere denne forudsigelsesfejl på enorme mængder tekst, ender modellerne med at lære begreber, der er nyttige for disse forudsigelser. For eksempel lærer de begreber som"(1):

* hvordan man staver
* hvordan grammatik fungerer
* hvordan man omformulerer
* hvordan man besvarer spørgsmål
* hvordan man fører en samtale
* hvordan man skriver på mange sprog
* hvordan man koder
* osv.

#### Sådan styrer du en stor sprogmodel  
"Af alle input til en stor sprogmodel er tekstprompten langt den mest indflydelsesrige"(1).

Store sprogmodeller kan promptes til at producere output på flere måder:

Instruktion: Fortæl modellen, hvad du vil have
Færdiggørelse: Få modellen til at færdiggøre begyndelsen på det, du ønsker
Demonstration: Vis modellen, hvad du vil have, enten med:
Et par eksempler i prompten
Mange hundrede eller tusinder af eksempler i et finjusterings-træningsdatasæt"



#### Der er tre grundlæggende retningslinjer for at lave prompts:

**Vis og fortæl**. Gør det tydeligt, hvad du ønsker, enten gennem instruktioner, eksempler eller en kombination af begge. Hvis du vil have modellen til at rangere en liste af elementer i alfabetisk rækkefølge eller til at klassificere et afsnit efter stemning, så vis den, at det er det, du vil have.

**Brug kvalitetsdata**. Hvis du prøver at bygge en klassificering eller få modellen til at følge et mønster, så sørg for, at der er nok eksempler. Husk at korrekturlæse dine eksempler — modellen er som regel klog nok til at gennemskue basale stavefejl og give dig et svar, men den kan også antage, at det er med vilje, og det kan påvirke svaret.

**Tjek dine indstillinger.** Indstillingerne temperature og top_p styrer, hvor deterministisk modellen er, når den genererer et svar. Hvis du beder om et svar, hvor der kun er ét rigtigt svar, vil du gerne sætte disse lavt. Hvis du ønsker mere varierede svar, kan du sætte dem højere. Den største fejl folk laver med disse indstillinger er at tro, at de styrer "intelligens" eller "kreativitet".


Kilde: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Opsummer tekst  
#### Udfordring  
Opsummer tekst ved at tilføje et 'tl;dr:' i slutningen af et tekstafsnit. Bemærk, hvordan modellen forstår at udføre en række opgaver uden yderligere instruktioner. Du kan eksperimentere med mere beskrivende prompts end tl;dr for at ændre modellens adfærd og tilpasse den opsummering, du får (3).  

Nylig forskning har vist markante fremskridt på mange NLP-opgaver og benchmarks ved at fortræne på et stort tekstkorpus og derefter finjustere på en specifik opgave. Selvom denne metode typisk er opgave-agnostisk i arkitekturen, kræver den stadig opgavespecifikke finjusteringsdatasæt med tusindvis eller titusindvis af eksempler. Til sammenligning kan mennesker generelt udføre en ny sproglig opgave ud fra blot få eksempler eller simple instruktioner – noget nuværende NLP-systemer stadig har svært ved. Her viser vi, at opskalering af sprogmodeller markant forbedrer opgave-agnostisk, få-skuds-præstation, og nogle gange endda når op på niveau med tidligere førende finjusteringsmetoder. 



Tl;dr


# Øvelser for forskellige brugsscenarier  
1. Opsummér tekst  
2. Klassificér tekst  
3. Find på nye produktnavne


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klassificér tekst  
#### Udfordring  
Klassificér elementer i kategorier, der gives ved inferenstidspunktet. I det følgende eksempel giver vi både kategorierne og teksten, der skal klassificeres, i prompten (*playground_reference).

Kundehenvendelse: Hej, en af tasterne på mit laptop-tastatur gik i stykker for nylig, og jeg har brug for en erstatning:

Klassificeret kategori:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Generér nye produktnavne
#### Udfordring
Skab produktnavne ud fra eksempler på ord. Her inkluderer vi i prompten information om det produkt, vi skal finde navne til. Vi giver også et lignende eksempel for at vise det mønster, vi ønsker at modtage. Vi har desuden sat temperaturværdien højt for at øge tilfældigheden og få mere innovative svar.

Produktbeskrivelse: En milkshake-maskine til hjemmet
Stikord: hurtig, sund, kompakt.
Produktnavne: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Produktbeskrivelse: Et par sko, der kan passe til enhver fodstørrelse.
Stikord: tilpasningsdygtig, pasform, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referencer  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Eksempler](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Bedste praksis for finjustering af GPT-3 til tekstklassificering](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# For mere hjælp  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Bidragydere
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiske oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel, menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå ved brug af denne oversættelse.
